In [1]:
import os
import time
from datetime import datetime

import matplotlib.pyplot as plt

from optuna.visualization import plot_param_importances, plot_optimization_history, plot_parallel_coordinate

import joblib


In [2]:
RESULTS_PATH = os.path.join(os.getcwd(), "training_results")

In [22]:
study = joblib.load('study.pkl')
model_trainer = joblib.load('model_trainer.pkl')

In [5]:
best_params = study.best_trial.params
print("Best parameters:", best_params)

Best parameters: {'batch_size': 128, 'lstm_units': 61, 'dropout_rate': 0.12089782674538327, 'learning_rate': 0.0001, 'num_lstm_layers': 1}


In [5]:
import optuna
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]

print(f"Trials completos: {len(complete_trials)}")
print(f"Trials podados (pruned): {len(pruned_trials)}")


Trials completos: 8
Trials podados (pruned): 17


In [19]:
def plot_and_save(study,plot_func, name, saving_path=None):
    """
    Generate and display a plot using the provided Optuna plotting function.
    Optionally saves the plot to a PNG file.

    Args:
        plot_func (callable): Optuna plotting function (e.g., plot_param_importances).
        name (str): Name to use for the saved plot file.
        saving_path (str or None): If provided, saves the plot to this directory.
    """
    import matplotlib.pyplot as plt
    
    try:
        fig = plot_func(study)
        fig.show()

        # Save figure if a path is provided
        if saving_path:
            os.makedirs(saving_path, exist_ok=True)
            fig_path = os.path.join(saving_path, f"{name}.png")
            plt.savefig(fig_path, format='png')

    except Exception as e:
        print(f"Error generating plot: {e}")
        raise e

In [21]:
saving_path = os.path.join(RESULTS_PATH, "plots")
# Create the directory if it doesn't exist
os.makedirs(saving_path, exist_ok=True)

# Generate and optionally save visualizations
plot_and_save(study,plot_param_importances, "param_importances")
plot_and_save(study,plot_optimization_history, "optimization_history.png")
plot_and_save(study,plot_parallel_coordinate, "parallel_coordinates.png")

In [6]:
print("\nCreating the best model...")
start = time.time()
best_model = model_trainer.create_model_architecture(best_params)
end = time.time()
print(f"Best model created successfully (took {(end - start) / 60:.2f} minutes)")


Creating the best model...


NameError: name 'model_trainer' is not defined

In [9]:
print("\nTraining the best model on full training set...")
start = time.time()
history = best_model.fit(
    model_trainer.X_train, model_trainer.y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=best_params["batch_size"],
    verbose=1
)
end = time.time()
print(f"Model trained successfully (took {(end - start) / 60:.2f} minutes)")


Training the best model on full training set...
Epoch 1/50
45155/45155 [==============================] - 494s 11ms/step - loss: 0.0026 - mae: 0.0249 - root_mean_squared_error: 0.0513 - val_loss: 1.3600e-05 - val_mae: 0.0020 - val_root_mean_squared_error: 0.0037
Epoch 2/50
45155/45155 [==============================] - 387s 9ms/step - loss: 2.2318e-04 - mae: 0.0105 - root_mean_squared_error: 0.0149 - val_loss: 9.1887e-06 - val_mae: 0.0018 - val_root_mean_squared_error: 0.0030
Epoch 3/50
45155/45155 [==============================] - 303s 7ms/step - loss: 1.9723e-04 - mae: 0.0097 - root_mean_squared_error: 0.0140 - val_loss: 3.8582e-05 - val_mae: 0.0052 - val_root_mean_squared_error: 0.0062
Epoch 4/50
45155/45155 [==============================] - 289s 6ms/step - loss: 1.8664e-04 - mae: 0.0094 - root_mean_squared_error: 0.0137 - val_loss: 9.0565e-06 - val_mae: 0.0023 - val_root_mean_squared_error: 0.0030
Epoch 5/50
45155/45155 [==============================] - 308s 7ms/step - loss: 1.

In [10]:
print("\nSaving the trained model...")
start = time.time()
best_model.save(os.path.join(RESULTS_PATH,'best_model.keras'))  # or "best_model.h5"
end = time.time()
print(f"Trained model saved successfully (took {(end - start) / 60:.2f} minutes)")


Saving the trained model...
Trained model saved successfully (took 0.00 minutes)


In [ ]:
# Make predictions on the test set


4.4046666666666665